In [1]:
import numpy as np
import pandas as pd
import random
import pulp

In [2]:
from google.colab import files
uploaded = files.upload()

import io
allrate = pd.read_csv(io.BytesIO(uploaded['allrate.csv']))

Saving allrate.csv to allrate.csv


In [2]:
allrate = pd.read_csv('../data/allrate.csv')
allrate[:9]

,Unnamed: 0,index,name,rate,title,rating,calories,sodium,fat,protein,personal_rating
0,0,0,rhaeredekop from Winnipeg,5.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."
1,1,0,lizgoldsmith1960 from Lincoln,5.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."
2,2,0,bmaybeegeorge from Canada,5.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."
3,3,0,adventurousz from Boulder,3.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."
4,4,0,epaul77 from Mississippi,5.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."
5,5,0,lizzydlangley from St Louis,5.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."
6,6,0,Jweyant from Maine,5.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."
7,7,0,tjsnuffin from Cincinnati,5.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."
8,8,0,sarahglittle7930 from Denver,5.0,"Curried Lentil, Tomato, and Coconut Soup",5.0,437.0,667.0,28.0,13.0,"[['rhaeredekop from Winnipeg, MB ', 5], ['lizg..."


## **Get nutrition matrix** 

In [3]:
nutrition = allrate.groupby(['index', 'title'])['calories', 'sodium', 'fat', 'protein'].agg('mean').reset_index()
nutrition[:10]

<ipython-input-3-683cf4f738c1>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  nutrition = allrate.groupby(['index', 'title'])['calories', 'sodium', 'fat', 'protein'].agg('mean').reset_index()


,index,title,calories,sodium,fat,protein
0,0,"Curried Lentil, Tomato, and Coconut Soup",437.0,667.0,28.0,13.0
1,1,Roasted Butternut Squash with Herb Oil and Goa...,175.0,576.0,9.0,4.0
2,2,Pumpkin Muffins,364.0,183.0,11.0,6.0
3,3,"Chopped Salad with Shallot Vinaigrette, Feta, ...",170.0,413.0,13.0,6.0
4,4,Grain Salad with Olives and Whole-Lemon Vinaig...,330.0,483.0,19.0,8.0
5,5,Chilled Coconut Corn Soup,481.0,496.0,36.0,8.0
6,6,Vietnamese-Style Spaghetti Squash “Noodle” Bow...,503.0,1574.0,34.0,29.0
7,8,Roasted Squash with Mint and Toasted Pumpkin S...,127.0,543.0,7.0,3.0
8,9,Butternut Squash Steaks with Brown Butter–Sage...,434.0,1661.0,19.0,6.0
9,10,Freeform Chicken Meatballs with Carrots and Yo...,488.0,985.0,32.0,29.0


In [4]:
# check the mising ratings
nutrition.isnull().sum()

index        0
title        0
calories    30
sodium      30
fat         30
protein     30
dtype: int64

In [5]:
# drop the mising ratings
nutrition2 = nutrition.dropna(axis = 0)
nutrition2

,index,title,calories,sodium,fat,protein
0,0,"Curried Lentil, Tomato, and Coconut Soup",437.0,667.0,28.0,13.0
1,1,Roasted Butternut Squash with Herb Oil and Goa...,175.0,576.0,9.0,4.0
2,2,Pumpkin Muffins,364.0,183.0,11.0,6.0
3,3,"Chopped Salad with Shallot Vinaigrette, Feta, ...",170.0,413.0,13.0,6.0
4,4,Grain Salad with Olives and Whole-Lemon Vinaig...,330.0,483.0,19.0,8.0
...,...,...,...,...,...,...
264,292,White Chicken Chili,534.0,968.0,14.0,45.0
265,293,One-Pot Curried Cauliflower with Couscous and ...,606.0,1365.0,15.0,27.0
267,295,Hummus Dinner Bowls with Spiced Ground Beef an...,329.0,327.0,26.0,20.0
268,296,Autumn Kale Salad,287.0,329.0,22.0,4.0


In [6]:
nutrition2.shape

(240, 6)

## **Rating matrix: $r$**

In [8]:
rating0 = pd.read_csv('../data/rating_with_group_member_predicted.csv')
title = nutrition2['title'].values.tolist()
rating = rating0[title]
rating

,"Curried Lentil, Tomato, and Coconut Soup",Roasted Butternut Squash with Herb Oil and Goat Cheese,Pumpkin Muffins,"Chopped Salad with Shallot Vinaigrette, Feta, and Dill",Grain Salad with Olives and Whole-Lemon Vinaigrette,Chilled Coconut Corn Soup,Vietnamese-Style Spaghetti Squash “Noodle” Bowls with Skirt Steak,Roasted Squash with Mint and Toasted Pumpkin Seeds,Butternut Squash Steaks with Brown Butter–Sage Sauce,Freeform Chicken Meatballs with Carrots and Yogurt Sauce,...,Sausage and Greens Sheet-Pan Dinner,Twice-Roasted Squash with Parmesan Butter and Grains,"Kale Salad with Butternut Squash, Pomegranate, and Pumpkin Seeds",Kabocha Squash Pilaf with Coconut,Chile-Marinated Pork with Vietnamese Brussels Sprouts,White Chicken Chili,One-Pot Curried Cauliflower with Couscous and Chickpeas,Hummus Dinner Bowls with Spiced Ground Beef and Tomatoes,Autumn Kale Salad,Pumpkin Icebox Pie With Snickerdoodle Crust
0,1,5,1,2,3,1,1,4,4,2,...,1,1,1,3,1,1,1,1,4,3
1,1,4,1,4,2,2,1,4,2,1,...,1,1,3,2,1,1,1,1,1,5
2,1,2,1,2,3,1,1,4,3,1,...,1,1,1,2,1,2,1,2,1,4
3,1,1,1,1,1,1,1,3,1,1,...,1,1,1,1,1,1,1,1,1,1


## **Allergy matrix: $a$** 

In [9]:
# get the dish list and the allergic food list
title = nutrition2['title'].tolist()
allergy_food = ['Shrimp', 'Pork', 'Shell', 'Fish']

# construct the allergy matrix
allergy_matrix = np.zeros((240, 240)).tolist()
for j in range(240):
    al = 0
    for food in allergy_food:
        if food in title[j]:
            al += 1
    if al == 0:
        allergy_matrix[j][j] = 1

In [10]:
# count the number of allergic food in the dish list
ll = []
for j in range(240):
    ll.append(allergy_matrix[j][j])

240 - sum(ll)

22.0

## **Price: $p$**


In [12]:
price = pd.read_csv('../data/pricing.csv')
price

,index,title,pricing
0,0,"Curried Lentil, Tomato, and Coconut Soup",1
1,1,Roasted Butternut Squash with Herb Oil and Goa...,2
2,2,Pumpkin Muffins,2
3,3,"Chopped Salad with Shallot Vinaigrette, Feta, ...",1
4,4,Grain Salad with Olives and Whole-Lemon Vinaig...,1
...,...,...,...
235,292,White Chicken Chili,4
236,293,One-Pot Curried Cauliflower with Couscous and ...,1
237,295,Hummus Dinner Bowls with Spiced Ground Beef an...,3
238,296,Autumn Kale Salad,1


In [41]:
# set the basic price of different level
# 1: vegetable & soup, 2: dessert & staple food, 3: meat & vegetables/staple food, 4: meat, 5: seafood
basic_price = {1: 12,
               2: 24,
               3: 28,
               4: 32,
               5: 60}

In [42]:
# generate a random price with basic price for each dish
random.seed(2)
r_price = []
for i in price['pricing']:
    r_price.append(basic_price[i] + random.randint(0, 600)/100)

price['price'] = r_price
price

,index,title,pricing,price
0,0,"Curried Lentil, Tomato, and Coconut Soup",1,12.57
1,1,Roasted Butternut Squash with Herb Oil and Goa...,2,24.93
2,2,Pumpkin Muffins,2,24.86
3,3,"Chopped Salad with Shallot Vinaigrette, Feta, ...",1,15.69
4,4,Grain Salad with Olives and Whole-Lemon Vinaig...,1,13.73
...,...,...,...,...
235,292,White Chicken Chili,4,33.03
236,293,One-Pot Curried Cauliflower with Couscous and ...,1,13.07
237,295,Hummus Dinner Bowls with Spiced Ground Beef an...,3,31.13
238,296,Autumn Kale Salad,1,14.03


## **Avaivability matrix: $Z$**

In [11]:
# construct the availability matrix
availability_matrix = [[1,1,0,1,1,1,0,1,1,1],
                       [1,0,0,1,1,1,0,1,1,1],
                       [1,1,0,1,1,1,0,1,1,1],
                       [1,1,0,1,1,1,0,1,1,1]]

## **Cooking ability matrix: $M$**

In [15]:
# construct the cooking ability matrix
cooking_ability_matrix = np.ones((4, 240)).tolist()

# Tinky-Winky can only cook ['Pasta', 'Noodle', 'Noodles', 'Burger', 'Salad']
for j in range(240):
    cooking_ability_matrix[0][j] = 0
    for food in ['Pasta', 'Noodle', 'Noodles', 'Burger', 'Salad']:
        if food in title[j]:
            cooking_ability_matrix[0][j] = 1

# Dipsy can not cook ['Beef', 'Lamb']
for j in range(240):
    for food in ['Beef', 'Lamb']:
        if food in title[j]:
            cooking_ability_matrix[1][j] = 0

# Laa-Laa can not cook ['cake', 'Pie']
for j in range(240):
    for food in ['cake', 'Pie']:
        if food in title[j]:
            cooking_ability_matrix[2][j] = 0

# Po can not cook ['Shrimp', 'Shell', 'Fish']
for j in range(240):
      for food in ['Shrimp', 'Shell', 'Fish']:
        if food in title[j]:
            cooking_ability_matrix[3][j] = 0

## **Basic Model**

In [55]:
# set up the optimization problem
meals = pulp.LpProblem("Meal_Plan", pulp.LpMaximize)

In [56]:
# the number of meals
meal = list(range(10))
nm = len(meal)

# the number of Teletubbies
Teletubby = ['Tinky-Winky', 'Dipsy', 'Laa-Laa', 'Po']
ns = len(Teletubby)

# the number of dishes
dish = nutrition2['title'].tolist()
nd = len(dish)

In [57]:
# set the variable

# d[j][k] --- whether dish j is selected in meal k
d = pulp.LpVariable.dicts("dish", (dish, meal), 0, 1, pulp.LpInteger)

# y[i][k] --- whether Teletubby i cooks in meal k
y = pulp.LpVariable.dicts("cook", (Teletubby, meal), 0, 1, pulp.LpInteger)

In [58]:
# set up the parameters
# rating
r = rating.values.tolist()
r = pulp.makeDict([Teletubby, dish], r, 0)

# nutrition
n_sodium = nutrition2['sodium'].tolist()
n_sodium = pulp.makeDict([dish], n_sodium, 0)
up_sodium = 2300 * 4 
low_sodium = 0

n_calory = nutrition2['calories'].tolist()
n_calory = pulp.makeDict([dish], n_calory, 0)
up_calory = 2000 * 4 
low_calory = 0

n_fat = nutrition2['fat'].tolist()
n_fat = pulp.makeDict([dish], n_fat, 0)
up_fat = 35 * 4 
low_fat = 25 * 4 

n_protein = nutrition2['protein'].tolist()
n_protein = pulp.makeDict([dish], n_protein, 0)
up_protein = 46 * 4 
low_protein = 0

# price and budget
p = price['price'].values.tolist()
p = pulp.makeDict([dish], p, 0)
budget = 300

# allergy
a = allergy_matrix
a = pulp.makeDict([dish, dish], a, 0)

# availability 
z = availability_matrix
z = pulp.makeDict([Teletubby, meal], z, 0)


# cooking ability --- M[i][j] means whehter student i can cook dish j
M = cooking_ability_matrix
M = pulp.makeDict([Teletubby, dish], M, 0)

In [59]:
# objective function
meals += (pulp.lpSum([r[i][j] * d[j][k] for i in Teletubby for k in meal for j in dish]), 'sum_of_rating')

In [60]:
# constraits

# the number of dishes in every meal --- no more than 3 dishes per meal 
for k in meal:
    meals += (pulp.lpSum([d[j][k] for j in dish]) <= 3)

# we eat the same dish at most twice
for j in dish:
    meals += (pulp.lpSum([d[j][k] for k in meal]) <= 2)

In [61]:
# nutrition
for k in [1, 3, 5, 7, 9]:
    # sodium
    meals += (pulp.lpSum([n_sodium[j] * d[j][k - 1] + n_sodium[j] * d[j][k] for j in dish]) <= up_sodium)
    meals += (pulp.lpSum([n_sodium[j] * d[j][k - 1] + n_sodium[j] * d[j][k] for j in dish]) >= low_sodium)
    
    # calory
    meals += (pulp.lpSum([n_calory[j] * d[j][k - 1] + n_calory[j] * d[j][k] for j in dish]) <= up_calory)
    meals += (pulp.lpSum([n_calory[j] * d[j][k - 1] + n_calory[j] * d[j][k] for j in dish]) >= low_calory)
    
    # fat
    meals += (pulp.lpSum([n_fat[j] * d[j][k - 1] + n_fat[j] * d[j][k] for j in dish]) <= up_fat)
    meals += (pulp.lpSum([n_fat[j] * d[j][k - 1] + n_fat[j] * d[j][k] for j in dish]) >= low_fat)
        
    # protein
    meals += (pulp.lpSum([n_protein[j] * d[j][k - 1] + n_protein[j] * d[j][k] for j in dish]) <= up_protein)
    meals += (pulp.lpSum([n_protein[j] * d[j][k - 1] + n_protein[j] * d[j][k] for j in dish]) >= low_protein)

In [62]:
# budget and price
for k in meal:
    meals += (pulp.lpSum([p[j] * d[j][k] for j in dish]) <= budget)

# allergy
for k in meal:
    for j in dish:
        meals += (a[j][j] * d[j][k] == d[j][k])

In [63]:
# cooking times constraints of y with z:
# the Teletubbies who cook at meal k must be available
for i in Teletubby:
    for k in meal:
        meals += (y[i][k] <= z[i][k])

# each meal will be cooked by no more than 2 Teletubbies
for k in meal:
    meals += (pulp.lpSum([y[i][k] for i in Teletubby]) <= 2)

# the Teletubbies will cook 3~4 meals in total
for i in Teletubby:
    meals += (pulp.lpSum([y[i][k] for k in meal]) <= 4)
    meals += (pulp.lpSum([y[i][k] for k in meal]) >= 3)


# ability of cooking
for k in meal:
    for j in dish:
        meals += (d[j][k] <= pulp.lpSum([y[i][k] * M[i][j] for i in Teletubby]))

In [64]:
# solve the problem
status = meals.solve()

In [65]:
# the optimizaiton objective function value is printed to the screen    
print("Total Rating Score = ", pulp.value(meals.objective))
print('----------------------------')
print('-------------------------------------------------------------------\n')

# print the dishes selected for each meal and Teletubbies who cook for each meal
weekday = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday'}
for k in meal:
    # print weekday and lunch/dinner
    if k%2 == 0:
        w = weekday[k/2]
        print(w + ' Lunch:')
    else:
        w = weekday[(k-1)/2]
        print(w + ' Dinner:')

    # print cooking Teletubby and menu
    ls_cook = []
    ls_dish = []
    for v in meals.variables():
        if (str(v).startswith('cook')) & (str(v).endswith(str(k))) & (v.varValue == 1):
            ls_cook.append(str(v)[:-2][5:])
        if (str(v).startswith('dish')) & (str(v).endswith(str(k))) & (v.varValue == 1):
            ls_dish.append(str(v)[:-3][5:])
            
    if ls_cook == []:
        print('No one is available! Eat outside!') # if all Teletubbies have class during the cooking time, eat outside
    elif ls_dish == []:
        print('The meal does not meet the nutrition requirements! Eat outside!') # if all Teletubbies who is available can not cook the dishes selected
    else:
        print('Teletubbies who cook: ' + str(ls_cook))
        print('Dishes: ')
        for i in ls_dish:
            print(i)

    print('-------------------------------------------------------------------\n')


Total Rating Score =  348.0
----------------------------
-------------------------------------------------------------------

Monday Lunch:
Teletubbies who cook: ['Po']
Dishes: 
French_Spiced_Bread
Quick_Sesame_Chicken_with_Broccoli
Sheet_Pan_Steak_Fajitas
-------------------------------------------------------------------

Monday Dinner:
Teletubbies who cook: ['Laa_Laa', 'Tinky_Winky']
Dishes: 
Honeynut_Squash_with_Radicchio_and_Miso
Roasted_Squash_with_Mint_and_Toasted_Pumpkin_Seeds
Sheet_Pan_Steak_Fajitas
-------------------------------------------------------------------

Tuesday Lunch:
No one is available! Eat outside!
-------------------------------------------------------------------

Tuesday Dinner:
Teletubbies who cook: ['Laa_Laa', 'Tinky_Winky']
Dishes: 
Butternut_Squash_Sandwich_with_Cheddar_Cheese_and_Pickled_Red_Onion
Easy_Green_Curry_with_Chicken,_Bell_Pepper,_and_Sugar_Snap_Peas
Shredded_Chicken_Salad_with_Creamy_Miso_Dressing
--------------------------------------------

In [54]:
pulp.LpStatus[meals.status]

'Optimal'

In [69]:
original = pd.read_csv('../data/rating_with_group_member.csv')
original

,Unnamed: 0,"Curried Lentil, Tomato, and Coconut Soup",Roasted Butternut Squash with Herb Oil and Goat Cheese,Pumpkin Muffins,"Chopped Salad with Shallot Vinaigrette, Feta, and Dill",Grain Salad with Olives and Whole-Lemon Vinaigrette,Chilled Coconut Corn Soup,Vietnamese-Style Spaghetti Squash __oodle_ Bowls with Skirt Steak,Roasted Squash with Mint and Toasted Pumpkin Seeds,Butternut Squash Steaks with Brown Butter__age Sauce,...,Twice-Roasted Squash with Parmesan Butter and Grains,"Kale Salad with Butternut Squash, Pomegranate, and Pumpkin Seeds",Kabocha Squash Pilaf with Coconut,Chile-Marinated Pork with Vietnamese Brussels Sprouts,White Chicken Chili,One-Pot Curried Cauliflower with Couscous and Chickpeas,Acorn Squash with Kale and Sausage,Hummus Dinner Bowls with Spiced Ground Beef and Tomatoes,Autumn Kale Salad,Pumpkin Icebox Pie With Snickerdoodle Crust
0,Tinky-Winky,0,0,0,0,0,0,0,4,0,...,0,1,0,0,0,0,0,0,0,0
1,Dipsy,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
2,Laa-Laa,0,0,0,2,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,4
3,Po,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
predicted = pd.read_csv('../data/rating_with_group_member_predicted.csv')
predicted

,Unnamed: 0,"Curried Lentil, Tomato, and Coconut Soup",Roasted Butternut Squash with Herb Oil and Goat Cheese,Pumpkin Muffins,"Chopped Salad with Shallot Vinaigrette, Feta, and Dill",Grain Salad with Olives and Whole-Lemon Vinaigrette,Chilled Coconut Corn Soup,Vietnamese-Style Spaghetti Squash __oodle_ Bowls with Skirt Steak,Roasted Squash with Mint and Toasted Pumpkin Seeds,Butternut Squash Steaks with Brown Butter__age Sauce,...,Twice-Roasted Squash with Parmesan Butter and Grains,"Kale Salad with Butternut Squash, Pomegranate, and Pumpkin Seeds",Kabocha Squash Pilaf with Coconut,Chile-Marinated Pork with Vietnamese Brussels Sprouts,White Chicken Chili,One-Pot Curried Cauliflower with Couscous and Chickpeas,Acorn Squash with Kale and Sausage,Hummus Dinner Bowls with Spiced Ground Beef and Tomatoes,Autumn Kale Salad,Pumpkin Icebox Pie With Snickerdoodle Crust
0,Tinky-Winky,1,5,1,2,3,1,1,4,4,...,1,1,3,1,1,1,1,1,4,3
1,Dipsy,1,4,1,4,2,2,1,4,2,...,1,3,2,1,1,1,1,1,1,5
2,Laa-Laa,1,2,1,2,3,1,1,4,3,...,1,1,2,1,2,1,1,2,1,4
3,Po,1,1,1,1,1,1,1,3,1,...,1,1,1,1,1,1,1,1,1,1
